### **Tune Network Stack (Buffers Size) To Increase Networking Performance**
https://www.cyberciti.biz/faq/linux-tcp-tuning/

### **Check status of services and store into csv file**
https://www.tecmint.com/list-all-running-services-under-systemd-in-linux/

In [94]:
from subprocess import check_output
from re import search
import csv

cmd = 'systemctl --type=service'
services = check_output(cmd.split()).decode().splitlines()[1:]
status = []

for i in services:
    match = search(r'(^.*)loaded.*(running|exited|failed)',i)
    if match:
        status.append(list(match.groups()))

status = sorted(status, key = lambda x: x[1], reverse = True)

with open('/home/mana/Work/status.csv','w') as file:
    writer = csv.writer(file)
    writer.writerows(status)

### **How to list of tcp services and port number and<br></br>store into csv file ?**

In [1]:
%%python3

from pathlib import Path
from re import search
import csv

file_read = Path('/etc/services').read_text().splitlines()
services = [i.replace('\t', ' ').rpartition('/tcp')[0].split()
        for i in file_read if search(r'/tcp', i)]

with open('services.csv', 'w', newline = '') as file:
    fieldnames = ['Services', 'Port']
    wrih = csv.DictWriter(file, fieldnames)
    wrih.writeheader()
    wrir = csv.writer(file, delimiter = ',', 
                      quoting = csv.QUOTE_NONE)
    wrir.writerows(services)    

### **Linux Error Code**

**Please help me in this script. I need to backup MySQL database.<br></br> On failure it must issue a notification mail. The below script is working. <br></br>But in every situation it sends success notification <br></br>only and issues only success state.**

**Use Python mysqldb Module**

!/bin/bash
<pre>
## Script:
# Backup script to take mysqldump
# Retain 7 days of backup
# Move backup to offsite
# Send success/failure email
# set up all the mysqldump variables
FILE=database.sql.`date +"%d-%m-%Y_%H.%M"`.gz
DBSERVER=127.0.0.1
#REMOTE_SERVER=127.0.0.1 # move database to offsite for backup
DATABASE=0000
DBUSER=00000
PASS=000
SRC_DIR="/home/admin/.."
#REMOTE_BACKUP_DIR=/root/mysql/backup
NOW=`date +"%d-%m-%Y_%H.%M"`
email_list="0@gamil.com"
success_email="0@gamil.com"
failure_email="0@gamil.com";
START=$(date +%s)
# function to send backup status email
sendEmail() {
scripttime=0;
END=$(date +%s)
DIFF=$(( $END - $START ))
if [ $DIFF -le 60 ]; then
scripttime="$DIFF seconds.";
else
DIFF=$(( $DIFF / 60 ))
scripttime="$DIFF minutes.";
fi;
content="$content. Log: Backup duration: $scripttime"
echo $content | mail -s "$subject" $email_list
exit;
}
# pipeline will return failure code if the mysqldump command fails
set -o pipefail
# Taking backup of all databases
mysqldump --user=${DBUSER} --password=${PASS} --default-character-set=utf8 ${DATABASE} | gzip > ${SRC_DIR}/${FILE} 2>/dev/null
# Verify backup is success/failure
RESULT=$?
if [ $RESULT -ne 0 ]; then
subject="Backup-FAILURE";
content="Backup appears to have been failed for $NOW. The mysqldump/image backup command returned failure status. Please login to $DBSERVER and check the status."
email_list=$failure_email
echo "[`date`]Backup failure."
sendEmail
fi
# Keep only last 7 days worth backup on database server.
find $SRC_DIR -mtime +7 -name '*.gz' -exec rm {} \;
# Finally send successful backup completion email.
subject="Backup-SUCCESS for on $NOW"
content="The backup file of database is on $SRC_DIR/$FILE and the backup of image is on $SRC_IMAGE_DIR/$IMG_FILE "
content=$content.`ls -lhtr $SRC_DIR/ | awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,"\r"}'`.`ls -lhtr $SRC_IMAGE_DIR/ | awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,"\r"}'`
email_list=$success_email;
echo "[`date +"%H:%M"`]Backup Success."
sendEmail
exit 0;
</pre>